In [1]:
import numpy as np
import os
import glob
import joblib
from tqdm import tqdm

from nilearn.input_data import NiftiMasker
import nibabel as nib

import llms_brain_lateralization as lbl
from llms_brain_lateralization import make_dir, standardize

In [2]:
subject_list = np.sort(glob.glob(os.path.join(lbl.fmri_data_resampled, 'sub-EN*')))

fmri_subs_runs = []
for sub_id in tqdm(subject_list):
    sub_id_basename = os.path.basename(sub_id)
    fmri_imgs_sub = sorted(glob.glob(os.path.join(sub_id, '*.nii.gz')))
    fmri_runs = [] # n_runs x n_timesteps x n_voxels
    for fmri_img in fmri_imgs_sub:
        nifti_masker = NiftiMasker(mask_img='mask_lpp_en.nii.gz', detrend=True, standardize=True,
                                   high_pass=1/128, t_r=lbl.t_r)
        fmri_runs.append(nifti_masker.fit_transform(fmri_img))
    fmri_subs_runs.append(fmri_runs)

100%|████████████████████████████████████████████████████████████████| 48/48 [14:10<00:00, 17.71s/it]


In [3]:
make_dir(lbl.fmri_data_avg_subject)

In [4]:
for run in range(lbl.n_runs):
    fmri_mean_sub = np.mean([fmri_sub_runs[run] for fmri_sub_runs in fmri_subs_runs], axis=0)
    fmri_mean_sub = standardize(fmri_mean_sub, axis=0)
    filename = os.path.join(lbl.fmri_data_avg_subject, 'average_subject_run-{}.gz'.format(run))
    with open(filename, 'wb') as f:
         joblib.dump(fmri_mean_sub, f, compress=4)

In [5]:
# now compute reliable voxels
from sklearn.linear_model import Ridge
import time

np.random.seed(1234)

n_subjects = len(subject_list)
n_voxels = nifti_masker.n_elements_

alphas = np.logspace(2,7,16)

n_trials = 10

corr_split = []
for i_trial in range(n_trials):
    print('='*80)
    
    idx_random = np.arange(n_subjects)
    np.random.shuffle(idx_random)
    
    idx_group_1 = idx_random[:n_subjects//2]
    idx_group_2 = idx_random[n_subjects//2:]
    
    regressors_runs = [np.mean([fmri_subs_runs[idx_sub][run][10:-10] for idx_sub in idx_group_1], axis=0)
                                   for run in range(lbl.n_runs)]
    fmri_runs = [np.mean([fmri_subs_runs[idx_sub][run][10:-10] for idx_sub in idx_group_2], axis=0)
                                   for run in range(lbl.n_runs)]

    corr_runs = []
    for run_test in range(lbl.n_runs):
        tic = time.time()
        
        runs_train = np.setdiff1d(np.arange(lbl.n_runs), run_test)
        x_train = np.vstack([regressors_runs[run_train] for run_train in runs_train])
        x_test = regressors_runs[run_test]
        y_train = np.vstack([fmri_runs[run_train] for run_train in runs_train])
        y_test = fmri_runs[run_test]
        
        ############ start nested CV 
        #leave another run apart as a validation test
        run_val = runs_train[0]
        runs_train_val = np.setdiff1d(runs_train, run_val)
        x_train_val = np.vstack([regressors_runs[run_train_val] for run_train_val in runs_train_val])
        x_val = regressors_runs[run_val]
        y_train_val = np.vstack([fmri_runs[run_train] for run_train in runs_train_val])
        y_val = fmri_runs[run_val]

        corr_val = []
        for alpha in alphas:
            model = Ridge(alpha=alpha, fit_intercept=False)
            model.fit(x_train_val, y_train_val)
            y_pred = model.predict(x_val)
            corr_tmp = [np.corrcoef(y_val[:,i], y_pred[:,i])[0,1] for i in range(n_voxels)]
            corr_val.append(corr_tmp)    

        idx_best_alpha = np.argmax(np.mean(corr_val, axis=1))
        alpha = alphas[idx_best_alpha]
        ############ end nested CV 
        
        model = Ridge(alpha=alpha, fit_intercept=False)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        corr_tmp = [np.corrcoef(y_test[:,i], y_pred[:,i])[0,1] for i in range(n_voxels)]

        corr_runs.append(corr_tmp)
        
        toc = time.time()
        
        print('run ', run_test, '\t', 'mean = {:.03f}'.format(np.mean(corr_tmp)), '\t',
            'max = {:.03f}'.format(np.max(corr_tmp)), '\t',
            'time elapsed = {:.03f}'.format(toc-tic))

    corr_split.append(np.mean(corr_runs, axis=0))

run  0 	 mean = 0.382 	 max = 0.872 	 time elapsed = 62.829
run  1 	 mean = 0.304 	 max = 0.838 	 time elapsed = 61.730
run  2 	 mean = 0.340 	 max = 0.873 	 time elapsed = 62.695
run  3 	 mean = 0.369 	 max = 0.894 	 time elapsed = 64.631
run  4 	 mean = 0.408 	 max = 0.913 	 time elapsed = 61.506
run  5 	 mean = 0.354 	 max = 0.867 	 time elapsed = 61.061
run  6 	 mean = 0.323 	 max = 0.851 	 time elapsed = 61.028
run  7 	 mean = 0.305 	 max = 0.887 	 time elapsed = 63.140
run  8 	 mean = 0.360 	 max = 0.875 	 time elapsed = 60.324
run  0 	 mean = 0.413 	 max = 0.887 	 time elapsed = 60.911
run  1 	 mean = 0.309 	 max = 0.801 	 time elapsed = 63.779
run  2 	 mean = 0.351 	 max = 0.818 	 time elapsed = 62.943
run  3 	 mean = 0.365 	 max = 0.872 	 time elapsed = 61.480
run  4 	 mean = 0.414 	 max = 0.899 	 time elapsed = 63.448
run  5 	 mean = 0.359 	 max = 0.823 	 time elapsed = 59.274
run  6 	 mean = 0.342 	 max = 0.854 	 time elapsed = 61.140
run  7 	 mean = 0.319 	 max = 0.869 	 ti

In [6]:
filename = 'corr_group_split_{}trials.gz'.format(n_trials)
with open(filename, 'wb') as f:
     joblib.dump(np.array(corr_split), f, compress=4)